In [ ]:
import sys
sys.path.append("../")  # add parent directory to the system path
from utils_data import *
from utils_model import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from time import time

In [ ]:
# get the data ready
root_name = "Research"
train_path, test_path, val_path = make_data_path(root_name=root_name)

DataGenerator = ImageDataGenerator(rescale=1.0 / 255)
train_data, test_data, val_data = make_data_ready(
    DataGenerator, train_path, test_path, val_path
)

In [ ]:
# ________________ CONFIG THE BASE MODELS ________________ #
weights = "imagenet"
input_shape = (224, 224, 3)
num_class = 524

include_top = False
base_trainable = False
pooling = "max"
learning_rate = 0.1
epochs = 1


model_funcs = [
    tf.keras.applications.inception_v3.InceptionV3,
    tf.keras.applications.resnet50.ResNet50,
    tf.keras.applications.mobilenet.MobileNet,
    tf.keras.applications.mobilenet_v2.MobileNetV2,
    tf.keras.applications.efficientnet.EfficientNetB0,
    tf.keras.applications.efficientnet_v2.EfficientNetV2B0,
]

model_0s = {}

for model_func in model_funcs:
    model_name = model_func.__name__

    model_0s[model_name] = configure_model_base(
        model_func=model_func,
        weights=weights,
        include_top=False,
        base_trainable=False,
        input_shape=input_shape,
        pooling=pooling,
        num_class=num_class,
        learning_rate=learning_rate,
    )

In [ ]:
# ________________ TEST TRAIN THE BASE MODELS ________________ #
history = {}
metrics = {}

for model_name, model_0 in model_0s.items():
    # train the models

    start_time = time()

    history[model_name] = model_0.fit(
        train_data,
        epochs=epochs,
        steps_per_epoch=len(train_data),
        validation_data=val_data,
        validation_steps=int(0.25 * len(val_data)),
    )

    end_time = time()
    execution_time = end_time - start_time

    # save metrics for the models
    metrics[model_name] = {
        "train_loss": history[model_name].history["loss"],
        "train_accuracy": history[model_name].history["accuracy"],
        "val_loss": history[model_name].history["val_loss"],
        "val_accuracy": history[model_name].history["val_accuracy"],
        "execution_time": execution_time,
    }

    # evaluate the model on test data and save the results
    test_loss, test_accuracy = model_0.evaluate(test_data)
    metrics[model_name]["test_loss"] = test_loss
    metrics[model_name]["test_accuracy"] = test_accuracy

In [ ]:
file_path = "../output/data"
file_name = "model_performance_log.json"
save_metric(metrics, file_name, file_path)

In [ ]:
file_path = "../output/data"
file_name = "model_performance_mobilenet_finetune2.json"

loaded_metrics = load_dict_from_json(file_name, file_path)

nrows = 6
ncols = 2
width = 10
height = nrows * width/ncols

figure = visualize_metric(loaded_metrics, nrows=nrows, ncols=ncols, figsize=(width, height))
# save_plot(figure, file_name, file_path)